# Trip Mapping - Itinerary Creation

### Dependencies and data

In [1]:
# Dependencies
import os
import gmaps
import pandas as pd
from config import GMAPS_KEY

In [2]:
# Configure API key
gmaps.configure(api_key=GMAPS_KEY)

In [3]:
# Hotel data
data_path = os.path.join('data', 'hotels.csv')
hotel_df = pd.read_csv(data_path)
hotel_df.head(2)

,Id,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
0,1,Vendas Novas,PT,38.6771,-8.4579,2021-04-18 15:28:30,few clouds,73.4,46,12.66,20,0,0,Hotel Acez
1,4,Cap Malheureux,MU,-19.9842,57.6142,2021-04-18 15:28:32,overcast clouds,77.0,84,3.00,100,0,0,Pereybere Hotel & Spa


### Narrow search and find 4-5 nearby cities

In [4]:
# Lowest and highest `Max Temp` in the data
min_mt, max_mt = hotel_df['Max Temp'].min(), hotel_df['Max Temp'].max()
min_mt, max_mt

(70.0, 79.93)

In [5]:
def user_input(prompt, min_temp=min_mt, max_temp=max_mt, binary=False):
    
    """
    Take user input for weather preferences.
    
    Parameters
    ----------
    prompt : str
        Input prompt
    min_temp : float, optional
        Minimum temperature in data, by default {min_mt}
    max_temp : float, optional
        Maximum temperature in data, by default {max_mt}
    binary : bool, optional
        Whether the input should be "yes" or "no" only, by default False
        
    Returns
    -------
    float
        User input
    """
    
    # Init user input and loop condition to False
    i = b = False
    
    # Get user input
    if binary: # yes/no input
        while i not in ['yes', 'no']: # continue until input is either "yes" or "no"
            i = input(prompt) # user input
        i = 1 if i == 'yes' else 0 # convert input to num
    else: 
        prompt += f'\tEnter a number between {min_temp} and {max_temp}: ' 
        while not b: # loop condition
            try:
                i = float(input(prompt)) # get user input as num
            except:
                continue
            if min_temp <= i <= max_temp: # check if user input is within proper temperature range
                b = True # end loop
            
    # Numeric input
    return i
        
    
# Get preferred weather
min_temp = user_input('1. What is the lowest temperature you would like at your destination?\n')
max_temp = user_input('2. What is the highest temperature you would like at your destination?\n', min_temp=min_temp)
rain = user_input('3. Would you like it to rain at your destination? (yes/no) ', binary=True)
snow = user_input('4. Would you like it to snow at your destination? (yes/no) ', binary=True)
min_temp, max_temp, rain, snow # enter 76, 78, no, no

1. What is the lowest temperature you would like at your destination?
	Enter a number between 70.0 and 79.93: 76
2. What is the highest temperature you would like at your destination?
	Enter a number between 76.0 and 79.93: 78
3. Would you like it to rain at your destination? (yes/no) no
4. Would you like it to snow at your destination? (yes/no) no


(76.0, 78.0, 0, 0)

In [6]:
# Filter for cities with the temp preference
pref_cities = hotel_df.loc[(hotel_df['Max Temp'] >= min_temp) & 
                           (hotel_df['Max Temp'] <= max_temp)]

# Filter rain preference
if rain:
    pref_cities = pref_cities.query('Rain > 0')
else:
    pref_cities = pref_cities.query('Rain == 0')

# Filter snow preference
if snow:
    pref_cities = pref_cities.query('Snow > 0')
else:
    pref_cities = pref_cities.query('Snow == 0')
    
pref_cities.shape

(61, 14)

### Temperature heatmap

In [7]:
# For Jupyterlab
# !jupyter labextension install @jupyter-widgets/jupyterlab-manager
# !jupyter lab build

In [8]:
# Locations and measurements
locations = pref_cities[['Latitude', 'Longitude']]
weights = pref_cities['Max Temp'].apply(lambda t: max(t, 0))

# Popup info box
hotel_info = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}, {Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
"""

# Layers
heatmap = gmaps.heatmap_layer(locations, weights, dissipating=False, point_radius=3, max_intensity=300)
markers = gmaps.marker_layer(locations,
                             info_box_content=[hotel_info.format(**row) for _, row in pref_cities.iterrows()])

# Heatmap
fig = gmaps.figure(center=(0.0, 0.0), zoom_level=1.5)
fig.add_layer(heatmap)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

### Create travel itinerary

In [9]:
# Select cities
cities = ['Miracema do Tocantins', 'Carutapera', 'Cabedelo', 'Pitimbu']
cities_df = pref_cities[pref_cities['City'].isin(cities)]
cities_df

,Id,City,Country,Latitude,Longitude,Time,Description,Max Temp,Humidity,Wind Speed,Cloudiness,Rain,Snow,Hotel
90,423,Cabedelo,BR,-6.9811,-34.8339,2021-04-18 15:31:24,broken clouds,77.00,94,5.75,75,0,0,Hotel Pigalle
104,483,Pitimbu,BR,-7.4706,-34.8086,2021-04-18 15:31:48,broken clouds,77.00,94,5.75,75,0,0,Reserva do Abiaí
105,488,Carutapera,BR,-1.1950,-46.0200,2021-04-18 15:31:50,moderate rain,77.61,92,4.45,100,0,0,Lidera
117,555,Miracema do Tocantins,BR,-9.5618,-48.3967,2021-04-18 15:32:17,overcast clouds,77.25,84,3.38,99,0,0,Miami Apart Hotel


In [10]:
# Extract coordinates for each city
cities_df = cities_df.loc[[117, 104, 105, 90]]
locations = cities_df[['Latitude', 'Longitude']].values
locations

array([[ -9.5618, -48.3967],
       [ -7.4706, -34.8086],
       [ -1.195 , -46.02  ],
       [ -6.9811, -34.8339]])

In [11]:
# Info
city_info = """
<dl>
<dt>Hotel</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}, {Country}</dd>
<dt>Weather</dt><dd>{Max Temp} F - {Description}</dd>
</dl>
"""

# Layers
directions = gmaps.directions_layer(start=locations[0], end=locations[1], 
                                    waypoints=locations[2:], show_markers=False)
markers = gmaps.marker_layer(locations,
                             info_box_content=[city_info.format(**row) for idx, row in cities_df.iterrows()])

# Map
fig = gmaps.figure(center=(-5.0, -60.0), zoom_level=5)
fig.add_layer(directions)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))